In [1]:
import cv2 
import numpy as np 
import os
from matplotlib import pyplot as plt 
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False 
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results  
    #손을 인식해서 좌표를 예측하는 메서드

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    #이미지에 점을 그리는 메서드 

In [5]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    while cap.isOpened():
    
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
    
        #draw_landmarks(image,results)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10)&0xFF == ord('a'):
            break
    cap.release()
    cv2.destroyAllWindows()#메서드 사용해봤다~

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh]) #키포인트 추출하는 메서드 (예측하기 쉽게 데이터를 정리한거야)

In [7]:
DATA_PATH = os.path.join('MP_Data') #학습데이터 저장하려고 파일경로 설정

actions = np.array(['yes', 'no', 'hate',"don't","i'm fine",'sorry',"Let's go",'do it','glad','good','bad','tired','follow you','follow me'])

no_sequences = 50 

sequence_length = 30 #한시퀀스에 30프레임

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
        #파일경로를 설정

In [22]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    action = actions[13]
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)
                
            if frame_num == 0:
                cv2.putText(image, 'STARTING COLLECTION', (120,200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collection frames for {} video Number {}'.format(action, sequence), (15,12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(2000)
            else:
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)
                
            if cv2.waitKey(10) & 0xFF == ord('a'):
                break
    cap.release()
    cv2.destroyAllWindows() 
    #학습데이터 만드는중, 데이터가 배열형식으로 저장됐다... 

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map #딕셔너리형식

{'yes': 0,
 'no': 1,
 'hate': 2,
 "don't": 3,
 "i'm fine": 4,
 'sorry': 5,
 "Let's go": 6,
 'do it': 7,
 'glad': 8,
 'good': 9,
 'bad': 10,
 'tired': 11,
 'follow you': 12,
 'follow me': 13}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        #학습데이터 불러오고 레이블링- 문제의 답을 알려주는거

In [13]:
np.array(sequences).shape #인풋(문제)데이터

(700, 30, 258)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(700, 30, 258)

In [16]:
y=to_categorical(labels).astype(int)

In [17]:
y[0]#Label과 신경망의 출력값을 비교하기 위해서 대응되도록 정수값 하나를 뉴런의 개수에 맞게 변환

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [41]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258))) #1.입력데이터와 입력층의 갯수를 맞춰줘야한다. 
#lstm 입력, 삭제, 출력게이트를 이용해서 셀상태 은닉상태를 계산한다.불필요한데이터지우고 오래 기억돼야할 데이터 정하는거
#rnn 과 비교했을때 긴 시퀀스의 입력을 처리하는데 좋음. 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))#마지막 레이어의 출력이 레이블이기때문에 (레이블이 모든 경우의수를 나타내기때문에)
#2.출력층의 갯수와 레이블의 갯수를 맞춰줘야한다.

In [42]:
model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [43]:
history=model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
21/21 [==============================] - 14s 219ms/step - loss: 2.6340 - categorical_accuracy: 0.1113
Epoch 2/200
21/21 [==============================] - 3s 124ms/step - loss: 2.5271 - categorical_accuracy: 0.1534
Epoch 3/200
21/21 [==============================] - 3s 125ms/step - loss: 2.2603 - categorical_accuracy: 0.1474
Epoch 4/200
21/21 [==============================] - 3s 123ms/step - loss: 2.2369 - categorical_accuracy: 0.1880
Epoch 5/200
21/21 [==============================] - 3s 128ms/step - loss: 2.0766 - categorical_accuracy: 0.1759
Epoch 6/200
21/21 [==============================] - 3s 125ms/step - loss: 2.0559 - categorical_accuracy: 0.1789
Epoch 7/200
21/21 [==============================] - 3s 125ms/step - loss: 1.9199 - categorical_accuracy: 0.1940
Epoch 8/200
21/21 [==============================] - 3s 126ms/step - loss: 2.1263 - categorical_accuracy: 0.1684
Epoch 9/200
21/21 [==============================] - 3s 125ms/step - loss: 2.0267 - categorical

21/21 [==============================] - 4s 180ms/step - loss: 0.1702 - categorical_accuracy: 0.9489
Epoch 144/200
21/21 [==============================] - 3s 127ms/step - loss: 0.1320 - categorical_accuracy: 0.9639
Epoch 145/200
21/21 [==============================] - 3s 128ms/step - loss: 0.1590 - categorical_accuracy: 0.9398
Epoch 146/200
21/21 [==============================] - 3s 129ms/step - loss: 0.4642 - categorical_accuracy: 0.8647
Epoch 147/200
21/21 [==============================] - 3s 130ms/step - loss: 0.5331 - categorical_accuracy: 0.7925
Epoch 148/200
21/21 [==============================] - 3s 133ms/step - loss: 0.2715 - categorical_accuracy: 0.9203
Epoch 149/200
21/21 [==============================] - 3s 133ms/step - loss: 0.1983 - categorical_accuracy: 0.9444
Epoch 150/200
21/21 [==============================] - 3s 130ms/step - loss: 0.1435 - categorical_accuracy: 0.9534
Epoch 151/200
21/21 [==============================] - 3s 130ms/step - loss: 0.2105 - categori

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[0])]

'do it'

In [29]:
actions[np.argmax(y_test[0])]

'do it'

In [30]:
res[0]

array([2.4979230e-04, 5.7187481e-07, 1.5276859e-05, 2.9834076e-03,
       2.4532428e-05, 3.5762271e-09, 3.0127123e-05, 9.9664491e-01,
       2.0850991e-06, 1.3881223e-14, 8.8510051e-14, 4.9232116e-05,
       5.3439982e-11, 2.9082910e-12], dtype=float32)

In [62]:
model.save('action1.h5')

In [63]:
del model

In [65]:
model.load_weights('action1.h5')

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [35]:
yhat = model.predict(X_train)

In [36]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [37]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[613,   4],
        [ 12,  36]],

       [[623,   0],
        [  1,  41]],

       [[603,  12],
        [ 10,  40]],

       [[617,   0],
        [  0,  48]],

       [[607,  12],
        [  6,  40]],

       [[615,   0],
        [  2,  48]],

       [[615,   3],
        [  0,  47]],

       [[619,   0],
        [  6,  40]],

       [[615,   1],
        [  0,  49]],

       [[613,   4],
        [  0,  48]],

       [[616,   1],
        [  1,  47]],

       [[618,   0],
        [  1,  46]],

       [[617,   0],
        [  0,  48]],

       [[615,   2],
        [  0,  48]]], dtype=int64)

In [38]:
accuracy_score(ytrue, yhat)

0.9413533834586466

In [44]:
sequence = []
sentence = []
threshold = 0.4
tmp = 0
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        #sequence.append(keypoints)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            if tmp != np.argmax(res):
                tmp = np.argmax(res)
                print(actions[np.argmax(res)])
            
        #if res[np.argmax(res)] > threshold:
         #   if len(sentence) > 0:
          #      if actions[np.argmax(res)] != sentence[-1]:
           #         sentence.append(actions[np.argmax(res)])
           # else:
            #    sentence.append(actions[0])
        
       # if len(sentence) > 5:
        #    sentence = sentence[-5:]
        
        #cv2.rectangle(image, (0,0),(640,40), (245,117,16), -1)
        #cv2.putText(image, ''.join(sentence), (3,30),
        #                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
        

tired
yes
do it
yes
tired
bad
don't
i'm fine
don't
good
i'm fine
Let's go
do it
hate
don't
i'm fine
do it
hate
do it
hate
do it
hate
don't
i'm fine
do it
hate
i'm fine
don't
bad
good
Let's go
do it
i'm fine
don't
good
Let's go
good
Let's go
i'm fine
good
i'm fine
Let's go
good
Let's go
good
i'm fine
good
follow you
glad
no
follow me
no
Let's go
good
follow you
glad
no
follow me
no
Let's go
good
Let's go
follow me
sorry
do it
hate
don't
bad
follow you
Let's go
good
Let's go
follow me
do it
i'm fine
good
don't
bad
follow you
no
glad
follow you
glad
no
Let's go
follow you
Let's go
no
glad
follow you
glad
no
glad
no
glad
no
follow me
sorry
do it
yes
tired
bad
follow you
glad
sorry
glad
no
glad
follow you
glad
no
follow me
Let's go
i'm fine
good
bad
follow you
glad
sorry
glad
no
glad
no
follow me
sorry
do it
hate
do it
hate
don't
hate
do it
hate
do it
yes
hate
tired
don't
good
i'm fine
do it
i'm fine
hate
i'm fine
do it
hate
i'm fine
hate
i'm fine
hate
do it
